## Аггрегация разметки датасета ruModAr

Аггрегация строится по следующей системе:

1. Сбор размеченных пулов с Толоки. Возможны варианты:
    - только общий пул нужно аггрегировать, тогда забирается только он
    - часть данных находится в контрольных заданиях и экзамене, тогда к основному пулу добавляются данные задания
2. Фильтрация разметчиков:
    - в общем пуле есть некоторое количество заранее размеченных заданий - контрольных
    - хорошим считается разметчик, который показывает `accuracy >= 0.5` на данных заданиях
    - формируется список "плохих" разметчиков
3. Аггрегация ответов разметчиков по заданиям:
    - форматирование в заданиях может отличаться от изначального из-за выгрузки с Толоки
    - учитываются только ответы "хороших" разметчиков
    - аггрегация по подготовленным пулам - создается массив карточек вида {key: value}, где key - кортеж из всех значимых элементов задания, value - список из кортежей вида (user_id, answer)
4. Голосование большинством по каждому заданию:
    - минимально необходимое большинство составляет 3 голоса, так как такое большинство валидно для перекрытия 5
    - по результату формируется датафрейм с заданиями и ответами
5. Подгрузка оригинальных данных с разметкой в виде таблицы с заданиями и ответами
6. Соединение таблиц:
    - очистка форматирования в таблице с ответами разметчиков и в таблице с правильными ответами
    - создание единых столбцов с полным заданием
    - соединение таблиц по данному столбцу
    - валидация размеров
7. Подсчет метрик

В разметке было 1800 заданий: по 600 заданий на каждую математическую операцию и по 300 на две опции:
- можно символ `->` заменить на просто `=`
- можно символ `->` заменить на ` + 1 =`

Разметчики делали выбор, и записывали свое решение. Проверяется не столько способность решить пример, сколько умение логически размышлять и делать вывод из контекста - уже решенных примеров.

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

### Сбор данных разметки и фильтрация разметчиков

Данный датасет представляет из себя лишь подмножество, которое было специально сгенерировано. Это не тестовый сет датасета из мультимодального бенчмарка (инструкционный сет)! Размер выборки - 1800 объектов.

In [2]:
assignments = pd.read_csv('assignments_from_pool_41461361__28-09-2023.tsv', sep='\t')
skills = pd.read_csv('workerSkills.csv', sep='|')

Разметчикам предлагалось на основании контекста из решенных пяти примеров и одного нерешенного примера ответить на вопрос, чему равен нерешенный пример, если заменить в нем специальный символ `->` соответственно контексту.
Вход: 
- INPUT:context (пример: `2 + 2 -> 5`).
- INPUT:problem (пример: `3 + 3 ->`).

Выход:
- OUTPUT:solution (целое число, например: `7`).

In [3]:
assignments.head(1)

,INPUT:context,INPUT:problem,OUTPUT:solution,GOLDEN:solution,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:task_suite_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started,ASSIGNMENT:submitted,ASSIGNMENT:accepted,ASSIGNMENT:reward
0,48 * 22 -> 1057\r\n7 * 23 -> 162\r\n56 * 68 ->...,80 * 35 ->,2801.0,NaN,NaN,NaN,https://platform.toloka.ai/task/41461361/00027...,000278a671--6514b53aaa6bb16340de1f8c,000278a671--651555040b89303759a5c4f3,000278a671--651555040b89303759a5c4f0,1d09bb1b80a54c03c195eb8af088f05b,APPROVED,2023-09-28T10:27:16.309,2023-09-28T10:31:26.669,2023-09-28T10:31:26.669,0.022


Приводим ответы к универсальному виду.

In [4]:
assignments['OUTPUT:solution'] = assignments['OUTPUT:solution'].astype(int)

Пока в ответах есть Nan'ы, нельзя сделать ответы интами, потому будут строками.

In [5]:
assignments['GOLDEN:solution'] = assignments['GOLDEN:solution'].apply(lambda x: x if str(x) == 'nan' else str(int(x)))

Фильтруем толокеров с `accuracy < 0.5` на контрольных заданиях, чтобы не учитывать их ответы при подсчете метрик.

In [6]:
from collections import defaultdict

users_dict = defaultdict(lambda: defaultdict(int))

for idx, row in assignments.iterrows():
    context = row[0]
    problem = row[1]

    out = row[2]
    
    gold = row[3]

    user = row[10]

    if str(user) != "nan" and str(gold) != "nan":
        if out == int(gold):
            users_dict[user]["good"] += 1
        else:
            users_dict[user]["bad"] += 1

print("Users total: ", len(users_dict))
bad_users = []
for key, value in users_dict.items():
    percentage_good = value["good"]/(value["good"] + value["bad"])
    if percentage_good < 0.5:
        bad_users.append(key)

print("Bad users:", len(bad_users))

Users total:  251
Bad users: 5


5 из 251 разметчиков на контрольных заданиях показали слишком плохое качество, чтобы учитывать их ответы для расчета метрики.

Отделяем контроль от основы, так как контрольные задания создавались отдельно и не должны учитываться при подсчете метрик. На контрольных заданиях есть `GOLDEN:solution`. Также отсеиваем возможные баги Толоки, когда в строке может не быть задания - `INPUT:context` содержит NaN.

In [7]:
assignments_no_control = assignments[assignments['GOLDEN:solution'].isnull()]
assignments_no_control_no_null = assignments_no_control[assignments_no_control['INPUT:context'].notnull()]

Посчитаем, сколько было затрачено на получение разметки тестовых данных без учета контрольных заданий, так как они могли проходиться неограниченное количество раз одним и тем же разметчиком.

In [8]:
def w_sum(df):
    idx = df.index.values
    vals = df.values
    summ = idx * vals
    return summ.sum()

d1 = assignments_no_control_no_null['ASSIGNMENT:reward'].value_counts(normalize=True)
d2 = assignments_no_control_no_null['ASSIGNMENT:reward'].value_counts()
print(f'взвешенная цена айтема в тесте: {round(w_sum(d1), 3)}')
print(f'потрачено на разметку теста: {round(w_sum(d2), 3)}')
print(f'{round(w_sum(d2), 3)} / {round(w_sum(d1), 3)}')

взвешенная цена айтема в тесте: 0.021
потрачено на разметку теста: 190.08
190.08 / 0.021


Выделим, сколько составила средняя часовая ставка для разметки тестовой части датасета. Это будет простое среднее из следующих величин: количество заданий, которое разметчк может сделать за час на основе данного задания, помноженное на цену задания.

In [9]:
def get_hour_pay(df):
    try:
        times = pd.to_datetime(df['ASSIGNMENT:submitted']) - pd.to_datetime(df['ASSIGNMENT:started'])
    except Exception as e:
        times = []
        for i in range(len(df)):
            try:
                start = pd.to_datetime(df['ASSIGNMENT:started'].iloc[i])
            except Exception as e:
                start = pd.to_datetime(df['ASSIGNMENT:started'].apply(lambda x: x.split('T')[1]).iloc[i])
            try:
                end = pd.to_datetime(df['ASSIGNMENT:submitted'].iloc[i])
            except Exception as e:
                start = pd.to_datetime(df['ASSIGNMENT:submitted'].apply(lambda x: x.split('T')[1]).iloc[i])
            delta = end - start
            times.extend([delta])
        times = pd.Series(times)
        # times = pd.to_datetime(df['ASSIGNMENT:submitted'].apply(lambda x: x.split('T')[1])) - pd.to_datetime(df['ASSIGNMENT:started'].apply(lambda x: x.split('T')[1]))
    sums = 3600 / times.apply(lambda x: x.seconds) * df['ASSIGNMENT:reward']
    return sums.mean()

get_hour_pay(assignments_no_control_no_null)

1.2339183264060753

### Сбор ответов разметчиков и голосование

Собираем ответы голосования большинством для каждого задания.

In [10]:
from collections import defaultdict

text_dict = defaultdict(list)

for context, problem, user, out in zip(
    assignments_no_control_no_null["INPUT:context"], assignments_no_control_no_null["INPUT:problem"], 
    assignments_no_control_no_null["ASSIGNMENT:worker_id"], assignments_no_control_no_null["OUTPUT:solution"]
    ):
    if user not in bad_users:
        text_dict[(context, problem)].append([
                user,
                {"out": out}
        ])

print(len(text_dict))

1800


In [11]:
keys = list(text_dict.keys())
Counter([len(text_dict[keys[i]]) for i in range(len(keys))])

Counter({5: 1774, 4: 26})

Есть 26 заданий, где перекрытие 4. Для формирования итоговых лейблов нужно, чтобы было простое большинство разметчиков, проголосовавших за данную опцию. Если большинства нет, то оценка строится, исходя из оценки навыков разметчиков. В таком случае, финальный лейбл будет присвоен по голосу группы с наилучшими навыками. Если по навыкам будет равенство, то решаем по ответам топ-3 по навыкам разметчиков. Если и данный способ дает равенство, то используются оценки навыков разметчиков из EM-алгоритма (реализация GLAD).

In [12]:
preds_full = {}
user2skill = {k:v for k, v in zip(skills['worker_id'], skills['skill_value'])}
control_acc = assignments[assignments['GOLDEN:solution'].notna()]\
    .groupby('ASSIGNMENT:worker_id')\
        .apply(lambda x: (np.array(x['OUTPUT:solution']) == np.array(x['GOLDEN:solution'])).mean())
user2control = {k:v for k, v in zip(control_acc.index, control_acc.values)}

from crowdkit.aggregation.classification.glad import GLAD

full = assignments['INPUT:context'] + ' ' + assignments['INPUT:problem']
id2task = dict(enumerate(full))
task2id = {k:v for v, k in id2task.items()}
id2user = dict(enumerate(assignments['ASSIGNMENT:worker_id']))
user2id = {k:v for v, k in id2user.items()}

codes = full.map(task2id)
res = pd.DataFrame({'task': codes, 'worker': assignments['ASSIGNMENT:worker_id'].map(user2id), 'label': assignments['OUTPUT:solution']})
model = GLAD(n_iter=1000, tol=1e-03, m_step_max_iter=1000, m_step_tol=1e-03)
model.fit(res)
user2alpha = dict(enumerate(model.alphas_))
tb = model.alphas_.copy()
tb.index = tb.index.map(id2user)
user2alpha = {k:v for k, v in zip(tb.index, tb.values)}

stats = {
    'total_agreement': 0,
    'majority': 0,
    'skill_based': 0,
    'major_based': 0,
    'em_based': 0,
    'rest': 0,
}

for i in range(len(keys)):
    ans = text_dict[keys[i]]
    lst = [[ans[j][0], ans[j][1]['out']] for j in range(len(ans))]
    users, votes = list(zip(*lst))
    cnt = pd.Series(Counter(votes)).sort_values(ascending=False)

    # # total agreement
    if len(cnt) == 1:
        res = cnt.index[0]
        stats['total_agreement'] += 1
    # simple majority
    elif cnt.iloc[0] > cnt.iloc[1]:
        res = cnt.index[0]
        stats['majority'] += 1
    # (> 1 options) & (1 option == 2 option)
    else:
        # try overall skill based comparison
        vals = list(map(lambda x: user2skill[x], users))
        table = pd.DataFrame({'user': users, 'votes': votes, 'skill': vals})
        agg = table.groupby('votes').agg(
            sum_skill=pd.NamedAgg(column='skill', aggfunc='sum'),
            sum_votes=pd.NamedAgg(column='user', aggfunc='count')
        ).sort_values(by=['sum_votes', 'sum_skill'], ascending=False)
        # check there is a leader by skills
        if agg['sum_skill'].iloc[0] > agg['sum_skill'].iloc[1]:
            res = agg.index[0]
            stats['skill_based'] += 1
        else:
            # top-3 answers by overall skills
            vals = list(map(lambda x: user2skill[x], users))
            table = pd.DataFrame({'user': users, 'votes': votes, 'skill': vals})
            table = table.sort_values(by='skill', ascending=False)
            if len(table) >= 3:
                sub = table.iloc[:3]
            else:
                sub = table
            agg = sub.groupby('votes').agg(
                sum_skill=pd.NamedAgg(column='skill', aggfunc='sum'),
                sum_votes=pd.NamedAgg(column='user', aggfunc='count')
            ).sort_values(by=['sum_votes', 'sum_skill'], ascending=False)
            if agg['sum_skill'].iloc[0] != agg['sum_skill'].iloc[1]:
                res = agg.index[0]
                stats['major_based'] += 1
            
            else:
                vals = list(map(lambda x: user2alpha[x], users))
                table = pd.DataFrame({'user': users, 'votes': votes, 'skill': vals})
                agg = table.groupby('votes').agg(
                    sum_skill=pd.NamedAgg(column='skill', aggfunc='sum'),
                    sum_votes=pd.NamedAgg(column='user', aggfunc='count')
                ).sort_values(by=['sum_votes', 'sum_skill'], ascending=False)
                # check there is a leader by skills
                if agg['sum_skill'].iloc[0] != agg['sum_skill'].iloc[1]:
                    res = agg.index[0]
                    stats['em_based'] += 1
                else:
                    res = agg.index[0]
                    stats['rest'] += 1

    preds_full[keys[i]] = res

C:\Users\artem\AppData\Roaming\Python\Python311\site-packages\crowdkit\aggregation\classification\glad.py:127: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[labels] = 0
C:\Users\artem\AppData\Roaming\Python\Python311\site-packages\crowdkit\aggregation\classification\glad.py:127: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[labels] = 0
C:\Users\artem\AppData\Roaming\Python\Python311\site-packages\crowdkit\aggregation\classification\glad.py:127: PerformanceWarning: DataFrame is highly fragmented.  This is usually the re

In [13]:
stats

{'total_agreement': 1439,
 'majority': 359,
 'skill_based': 2,
 'major_based': 0,
 'em_based': 0,
 'rest': 0}

In [14]:
preds_full_df = pd.concat([pd.DataFrame(preds_full.keys(), columns=['context', 'problem']), pd.DataFrame(preds_full.values(), columns=['lb'])], axis=1)

### Сопоставление разметки и ground truth

Забираем задания из датасета с правильными ответами.

In [15]:
res_df = pd.read_csv('exp_wa.tsv', sep='\t')

После скачивания с Толоки в текстах рушится форматирование, потому нельзя просто сделать join двух табличек. Нужно убрать все "лишнее" форматирование сразу из двух табличек, чтобы остались только тексты, пунктуация и пробелы.

In [16]:
def format_text(text):
    text = (text.strip().replace('\n', ' ').replace('\t', ' ')
            .replace('\r', ' ').replace('  ', ' ').replace('  ', ' ')
            .replace('  ', ' '))
    return text

res_df['INPUT:context'] = res_df['INPUT:context'].apply(format_text)
res_df['INPUT:problem'] = res_df['INPUT:problem'].apply(format_text)

preds_full_df['context'] = preds_full_df['context'].apply(format_text)
preds_full_df['problem'] = preds_full_df['problem'].apply(format_text)

res_df = res_df.rename({
    'INPUT:context': 'context',
    'INPUT:problem': 'problem',
    'outputs': 'lb'
}, axis=1)

res_df['full'] = res_df['context'] + ' ' + res_df['problem']
preds_full_df['full'] = preds_full_df['context'] + ' ' + preds_full_df['problem']

Делаем left join, чтобы соединить голосование и правильные метки для одних и тех же заданий.

In [17]:
new = res_df.merge(preds_full_df.drop(['context', 'problem'], axis=1), on='full', how='left')

In [18]:
new_valid = new[new['lb_y'].notna()].copy()
len(new_valid)

1800

Ни одна строка не была утеряна.

In [19]:
new_valid['lb_y'] = new_valid['lb_y'].astype(int)

In [20]:
new_valid.head(1)

,lb_x,context,problem,full,lb_y
0,874,199 + 998 -> 1197 191 + 519 -> 710 557 + 78 ->...,31 + 843 ->,199 + 998 -> 1197 191 + 519 -> 710 557 + 78 ->...,874


### Подсчет метрики

Если в правом столбце меток осталось 1800 непустых строк, значит, форматирование было подчищено корректно и ничего не потерялось

Попробуем посчитать разные метрики

In [21]:
(new_valid['lb_x'] == new_valid['lb_y']).mean()

0.9988888888888889

`Accuracy = 0.999`

### Анализ ошибок

Посмотрим на ошибки в голосовании большинством. Сначала добавим указание на части датасета.

In [22]:
(new_valid['lb_x'] != new_valid['lb_y']).sum()

2

Разметчики почти идеально решили задачу. Только в одном задании большинство ошиблось.

In [23]:
new_valid[new_valid['lb_x'] != new_valid['lb_y']]

,lb_x,context,problem,full,lb_y
439,1504,419 + 398 -> 818 80 + 955 -> 1036 881 + 712 ->...,918 + 585 ->,419 + 398 -> 818 80 + 955 -> 1036 881 + 712 ->...,1604
1164,-445,366 - 677 -> -310 797 - 428 -> 370 322 - 621 -...,25 - 471 ->,366 - 677 -> -310 797 - 428 -> 370 322 - 621 -...,-447


Очевидно, разметчики верно поняли, что тут специальный символ меняет результат на единицу, но перепутали и вместо прибавления единицы вычли ее. Больше анализировать в обработанных данных нечего - ошибок больше не осталось. Нужно смотреть на полный пул отдельных ответов разметчиковна каждое задание.

In [24]:
new = assignments_no_control_no_null.iloc[:, [0, 1, 2, 10]].rename({
    'INPUT:context': 'context',
    'INPUT:problem': 'problem',
    'OUTPUT:solution': 'label',
    'ASSIGNMENT:worker_id': 'user'
}, axis=1).copy()

Соединяем контекст и нерешенный пример, чтобы было удобно соединить с правильными ответами.

In [25]:
new['full'] = new.context + ' ' + new.problem
new['full'] = new['full'].apply(format_text)

In [26]:
new = new.merge(res_df.drop(['context', 'problem'], axis=1), on='full', how='left')

Соединяем с правильными ответами по полному тексту задания.

In [27]:
new['sign'] = new.problem.apply(lambda x: x.split()[1])

Посмотрим на разницу и абсолютную разницу ответов разметчиков и правильных ответов.

In [28]:
new['diff'] = new['label'] - new['lb']
new['abs_diff'] = new['diff'].abs()

Ошибками считаем те строки, где разность не равна нулю.

In [29]:
new['error'] = (new['diff'] != 0).astype(int)

In [30]:
new.head(1)

,context,problem,label,user,full,lb,sign,diff,abs_diff,error
0,48 * 22 -> 1057\r\n7 * 23 -> 162\r\n56 * 68 ->...,80 * 35 ->,2801,1d09bb1b80a54c03c195eb8af088f05b,48 * 22 -> 1057 7 * 23 -> 162 56 * 68 -> 3809 ...,2801,*,0,0,0


Отделим ошибки.

In [31]:
errors = new[new['error'] == 1].copy()

In [32]:
errors['diff'].value_counts()

diff
-2      116
-1       76
 1       25
 100      7
-3        5
       ... 
 504      1
 96       1
-30       1
 78       1
 640      1
Name: count, Length: 168, dtype: int64

Наиболее часто встречающаяся ошибка у толокеров, как и следовало ожидать, это просчет на -2. Природа этого просчета в простой арифметике. По заданию специальный символ может превращаться в `+ 1 =`, значит необходимо к результату вычисления прибавить единицу. В случае отрицательных чисел прибавление единицы число по модулю уменьшает. Отсюда и ошибка. Пример:

`-10 + 1 = -9`

`10 + 1 = 11`

А толокеры могут путать правила и делать так: 

`-10 + 1 = -11`

In [33]:
errors[errors['abs_diff'] == 2]['sign'].value_counts()

sign
-    89
+    17
*    11
Name: count, dtype: int64

Набиольшее количество таких ошибок допущенно в примерах с вычитанием, так как именно там много отрицательных чисел.

In [34]:
errors[(errors['abs_diff'] == 2) & (errors['lb'] < 0)]['sign'].value_counts()

sign
-    57
Name: count, dtype: int64

Что и требовалось доказать! Наиболее частая ошибка ярче всего проявляется в примерах с вычитанием. В большинстве случаев такая ошибка делается одним или двумя разметчиками, что не приводит к ее появлению при голосовании большинством. Однако в одном примере собралось сразу 3 разметчика, которые сделали одинаковую ошибку.

In [35]:
errors['sign'].value_counts()

sign
-    225
+    120
*     83
Name: count, dtype: int64

В целом, среди всех ошибок чаще всего разметчики ошибаются именно в примерах с вычитанием.

Теперь оценим вероятность ошибки грубо по всему пулу без заведомо "плохих" разметчиков.

In [36]:
pd.DataFrame({
    "Сложение": new[(new['error'] == 1) & (new['sign'] == '+')].shape[0] / len(new),
    "Вычитание": new[(new['error'] == 1) & (new['sign'] == '-')].shape[0] / len(new),
    "Умножение": new[(new['error'] == 1) & (new['sign'] == '*')].shape[0] / len(new),
}, index=['Доля ошибок от всего пула'])

,Сложение,Вычитание,Умножение
Доля ошибок от всего пула,0.013333,0.025,0.009222


Очевидно, вероятность ошибки на примере с вычитанием в 2 раза выше, чем на примере со сложением и почти в 3 раза больше, чем в примерах с умножением.